# UCSB Grades During the Pandemic

**Main focus:** How did the pandemic impact course grades at UCSB?

- Did grades during the pandemic differ from a year with no pandemic?
- Were there a higher proportion of passing letter grades given during the pandemic? Were there more people choosing P/NP?
- Which departments were more lenient with grading during the pandemic?
- Did grades change after resuming in-person instruction?

## Grades over time

In [1]:
import pandas as pd
import altair as alt

# read data
grades_raw = pd.read_csv('data/ucsb-grades-raw.csv')
grades_raw.head()

,Quarter,Course Level2,Course,Instructor,Grade Given,Sum of Student Count
0,S22,Undergraduate,ANTH 2,WALSH C,A,274
1,S22,Undergraduate,ANTH 2,WALSH C,A-,101
2,S22,Undergraduate,ANTH 2,WALSH C,A+,11
3,S22,Undergraduate,ANTH 2,WALSH C,B,37
4,S22,Undergraduate,ANTH 2,WALSH C,B-,13


In [2]:
# get markdown table
print(grades_raw.head().to_markdown())

|    | Quarter   | Course Level2   | Course      | Instructor   | Grade Given   |   Sum of Student Count |
|---:|:----------|:----------------|:------------|:-------------|:--------------|-----------------------:|
|  0 | S22       | Undergraduate   | ANTH      2 | WALSH C      | A             |                    274 |
|  1 | S22       | Undergraduate   | ANTH      2 | WALSH C      | A-            |                    101 |
|  2 | S22       | Undergraduate   | ANTH      2 | WALSH C      | A+            |                     11 |
|  3 | S22       | Undergraduate   | ANTH      2 | WALSH C      | B             |                     37 |
|  4 | S22       | Undergraduate   | ANTH      2 | WALSH C      | B-            |                     13 |


In [3]:
# get dimensions
grades_raw.shape

(403280, 6)

In [4]:
# define quarters of interest
quarters = ['F18', 'W19', 'S19', 'M19',
            'F19', 'W20', 'S20', 'M20',
            'F20', 'W21', 'S21', 'M21',
            'F21', 'W22', 'S22']

# quarters to match
quarters_pattern = '|'.join(quarters)

# filter quarters of interests and undergrad courses
grades_sub = grades_raw[(grades_raw['Quarter'].str.contains(quarters_pattern)) &
                        (grades_raw['Course Level2'] == 'Undergraduate')]

grades_sub

,Quarter,Course Level2,Course,Instructor,Grade Given,Sum of Student Count
0,S22,Undergraduate,ANTH 2,WALSH C,A,274
1,S22,Undergraduate,ANTH 2,WALSH C,A-,101
2,S22,Undergraduate,ANTH 2,WALSH C,A+,11
3,S22,Undergraduate,ANTH 2,WALSH C,B,37
4,S22,Undergraduate,ANTH 2,WALSH C,B-,13
...,...,...,...,...,...,...
138321,F18,Undergraduate,WRIT 199,FRANK D M,A+,1
138322,F18,Undergraduate,WRIT 199,GENOVA G L,A+,3
138323,F18,Undergraduate,WRIT 199,MORRIS N S,A+,2
138324,F18,Undergraduate,WRIT 199,MORRIS N S,A,2


In [5]:
# output file for filtered quarters
grades_sub.to_csv("data/ucsb-grades.csv", index = False)

In [6]:
# aggregate by quarter and grade using sum
grades_agg_sum = grades_sub \
    .groupby(['Quarter', 'Grade Given']) \
    .sum('Sum of Student Count') \
    .reset_index()

grades_agg_sum

,Quarter,Grade Given,Sum of Student Count
0,F18,A,23273
1,F18,A+,5962
2,F18,A-,13675
3,F18,B,14958
4,F18,B+,10983
...,...,...,...
224,W22,D+,651
225,W22,D-,242
226,W22,F,2701
227,W22,NP,75


Since the number of students taking each classes can differ between quarters (especially during the summer), proportions are used for standardization.

In [7]:
# aggregate by quarter and grade using proportion
grades_agg_prop = grades_sub[['Quarter', 'Grade Given', 'Sum of Student Count']] \
    .groupby(['Quarter', 'Grade Given']) \
    .sum('Sum of Student Count') \
    .groupby(level = 0) \
    .apply(lambda x: x / x.sum()) \
    .rename(columns = {'Sum of Student Count': 'Proportion of Students'}) \
    .reset_index()

grades_agg_prop

,Quarter,Grade Given,Proportion of Students
0,F18,A,0.224539
1,F18,A+,0.057522
2,F18,A-,0.131937
3,F18,B,0.144315
4,F18,B+,0.105964
...,...,...,...
224,W22,D+,0.007553
225,W22,D-,0.002808
226,W22,F,0.031338
227,W22,NP,0.000870


In [8]:
# join sum and proportion
grades_agg = grades_agg_sum.merge(
    grades_agg_prop, 
    on = ['Quarter', 'Grade Given']
)

grades_agg

,Quarter,Grade Given,Sum of Student Count,Proportion of Students
0,F18,A,23273,0.224539
1,F18,A+,5962,0.057522
2,F18,A-,13675,0.131937
3,F18,B,14958,0.144315
4,F18,B+,10983,0.105964
...,...,...,...,...
224,W22,D+,651,0.007553
225,W22,D-,242,0.002808
226,W22,F,2701,0.031338
227,W22,NP,75,0.000870


I originally wanted to add a line representing when COVID started (S20), but I could not figure out how to do it in the faceted plots below (the order of the quarters kept getting messed up).

In [9]:
# add missing quarters for IP grades
ip_missing = pd.DataFrame({'Quarter': ['F18', 'W19', 'S19', 'M19', 'S20', 'M20', 'S21', 'M21', 'F21', 'W22', 'S22'],
                           'Grade Given': ['IP', 'IP', 'IP', 'IP', 'IP', 'IP', 'IP', 'IP', 'IP', 'IP', 'IP'],
                           'Sum of Student Count': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           'Proportion of Students': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]})
grades_agg = grades_agg.append(ip_missing, ignore_index = True)

# define order of letter grades
letter_grades = ['A+', 'A', 'A-',
                 'B+', 'B', 'B-',
                 'C+', 'C', 'C-',
                 'D+', 'D', 'D-',
                 'F', 'IP', 'P', 'NP']

# line plots
lines = alt.Chart(grades_agg).mark_line().encode(
    x = alt.X('Quarter:O', sort = quarters),
    y = 'Proportion of Students',
    color = 'Grade Given:N'
)

# points
points = alt.Chart(grades_agg).mark_circle().encode(
    x = alt.X('Quarter:O', sort = quarters),
    y = 'Proportion of Students',
    color = alt.Color('Grade Given:N', legend = None)
)

# line plots for each letter grade over time
(lines + points).properties(
    width = 175,
    height = 150
).facet(
    facet = alt.Facet('Grade Given:N', sort = letter_grades),
    columns = 4
).resolve_scale(
    x = 'independent',
    y = 'independent'
)

alt.FacetChart(...)

## Department

There was an email sent out regarding which departments opted for P/NP in Winter 2021 we can use as reference.

**Notes:**

- Some departments' abbreviations have spaces, which causes the departments to be labeled by the first word (for example, "C LIT" to "C", "AS AM" to "AS"), so we need to fix that later.
- Some courses are offered online (e.g. PSTATW 120A), which means the the course is put into a different department. For example, the course PSTATW 120A is classified as being part of the PSTATW department (which is different from the PSTAT department).

In [10]:
# remove unusual spacing between course and course numbers
grades_sub = grades_sub.copy()
course_split = grades_sub['Course'].str.split()
grades_sub['Course'] = course_split.str.join(' ')

# create departments column
grades_sub['Department'] = grades_sub.loc[:, 'Course'].str.split().str[0]
grades_sub

,Quarter,Course Level2,Course,Instructor,Grade Given,Sum of Student Count,Department
0,S22,Undergraduate,ANTH 2,WALSH C,A,274,ANTH
1,S22,Undergraduate,ANTH 2,WALSH C,A-,101,ANTH
2,S22,Undergraduate,ANTH 2,WALSH C,A+,11,ANTH
3,S22,Undergraduate,ANTH 2,WALSH C,B,37,ANTH
4,S22,Undergraduate,ANTH 2,WALSH C,B-,13,ANTH
...,...,...,...,...,...,...,...
138321,F18,Undergraduate,WRIT 199,FRANK D M,A+,1,WRIT
138322,F18,Undergraduate,WRIT 199,GENOVA G L,A+,3,WRIT
138323,F18,Undergraduate,WRIT 199,MORRIS N S,A+,2,WRIT
138324,F18,Undergraduate,WRIT 199,MORRIS N S,A,2,WRIT


In [11]:
# get unique departments (some departments are overcounted because they have online versions for a few classes)
unique_dept = grades_sub['Department'].unique()
print("Number of unique academic departments at UCSB:", len(unique_dept))
unique_dept

Number of unique academic departments at UCSB: 77


array(['ANTH', 'ART', 'ARTHI', 'ARTHIW', 'AS', 'ASTRO', 'BIOL', 'BL', 'C',
       'CH', 'CHEM', 'CHIN', 'CLASS', 'CMPSC', 'CMPSCW', 'CMPTGCS',
       'CNCSP', 'COMM', 'DANCE', 'EACS', 'EARTH', 'ECE', 'ECON', 'ED',
       'EEMB', 'ENGL', 'ENGR', 'ENV', 'ES', 'ESS', 'FAMST', 'FEMST', 'FR',
       'GEOG', 'GER', 'GLOBL', 'GREEK', 'HIST', 'INT', 'ITAL', 'JAPAN',
       'KOR', 'LAIS', 'LATIN', 'LING', 'MAT', 'MATH', 'MATRL', 'MCDB',
       'ME', 'MS', 'MUS', 'PHIL', 'PHYS', 'POL', 'PORT', 'PSTAT', 'PSY',
       'RG', 'RUSS', 'SHS', 'SLAV', 'SOC', 'SPAN', 'THTR', 'TMP', 'W&L',
       'WRIT', 'CNCSPW', 'EARTHW', 'MES', 'PSTATW', 'DANCEW', 'FAMSTW',
       'FEMSTW', 'HEB', 'BMSE'], dtype=object)

In [12]:
# get grades by department
grades_by_dept = grades_sub.groupby(['Quarter', 'Grade Given', 'Department']) \
    .sum('Sum of Student Count') \
    .reset_index()

grades_by_dept

,Quarter,Grade Given,Department,Sum of Student Count
0,F18,A,ANTH,342
1,F18,A,ART,542
2,F18,A,ARTHI,418
3,F18,A,AS,162
4,F18,A,ASTRO,15
...,...,...,...,...
11079,W22,P,PHYS,100
11080,W22,P,PSTAT,5
11081,W22,P,SOC,8
11082,W22,P,TMP,212


In [13]:
# get number of students in each department during each quarter
dept_count = grades_sub.groupby(['Quarter', 'Department']) \
    .sum('Sum of Student Count') \
    .reset_index() \
    .rename(columns = {'Sum of Student Count': 'Number of Students in Department'})

dept_count

,Quarter,Department,Number of Students in Department
0,F18,ANTH,1869
1,F18,ART,1123
2,F18,ARTHI,1266
3,F18,AS,814
4,F18,ASTRO,259
...,...,...,...
976,W22,SPAN,799
977,W22,THTR,548
978,W22,TMP,628
979,W22,W&L,79


In [14]:
# merge the datasets to get proportion of students in each department
merged = grades_by_dept.merge(dept_count, on = ['Quarter', 'Department'])
merged['Proportion in Department'] = merged['Sum of Student Count'] / merged['Number of Students in Department']
merged

,Quarter,Grade Given,Department,Sum of Student Count,Number of Students in Department,Proportion in Department
0,F18,A,ANTH,342,1869,0.182986
1,F18,A+,ANTH,76,1869,0.040663
2,F18,A-,ANTH,265,1869,0.141787
3,F18,B,ANTH,292,1869,0.156233
4,F18,B+,ANTH,237,1869,0.126806
...,...,...,...,...,...,...
11079,W22,NP,ES,38,664,0.057229
11080,W22,P,ES,624,664,0.939759
11081,W22,P,BIOL,66,66,1.000000
11082,W22,P,CMPTGCS,36,36,1.000000


In [15]:
# filter for only A+, A, and A- grades
a_grades = merged[(merged['Grade Given'] == 'A') | 
                  (merged['Grade Given'] == 'A-') | 
                  (merged['Grade Given'] == 'A+')]

# line plots for each department
alt.Chart(a_grades).mark_line().encode(
    x = alt.X('Quarter:O', sort = quarters),
    y = 'Proportion in Department',
    color = 'Grade Given'
).facet(
    facet = alt.Facet('Department:N', sort = letter_grades),
    columns = 4
)

alt.FacetChart(...)

In [16]:
# distribution of grades for the PSTAT department
alt.Chart(merged[merged.Department == 'PSTAT']).mark_line().encode(
    x = alt.X('Quarter:O', sort = quarters),
    y = 'Proportion in Department',
    color = 'Grade Given'
)

alt.Chart(...)

How do we determine biggest rate of change in the distribution of grades?

For each department:

- Find the mean grade (possibly by converting letter grades to GPA) before COVID
- Find the mean grade during COVID
- Calculate the difference between these two
- See if there's a huge difference

## Test

In [17]:
# define quarters of interest
quarters = ['F18', 'W19', 'S19', 'M19',
            'F19', 'W20', 'S20', 'M20',
            'F20', 'W21', 'S21', 'M21',
            'F21', 'W22', 'S22']

In [18]:
# pre-pandemic grades for a particular department (similar process for during and post-pandemic)
grades_sub[(grades_sub['Quarter'].isin(quarters[:6])) &          # quarters before S20 (pandemic)
           (grades_sub['Department'] == 'ANTH') &                # department of interest
           (~grades_sub['Grade Given'].isin(['P', 'NP', 'IP']))] # exclude grades w/o GPA equivalent

,Quarter,Course Level2,Course,Instructor,Grade Given,Sum of Student Count,Department
70709,W20,Undergraduate,ANTH 2,WALSH C,A,136,ANTH
70710,W20,Undergraduate,ANTH 2,WALSH C,A-,89,ANTH
70711,W20,Undergraduate,ANTH 2,WALSH C,A+,17,ANTH
70712,W20,Undergraduate,ANTH 2,WALSH C,B,48,ANTH
70713,W20,Undergraduate,ANTH 2,WALSH C,B-,21,ANTH
...,...,...,...,...,...,...,...
125381,F18,Undergraduate,ANTH 199RA,FORD A,C+,1,ANTH
125382,F18,Undergraduate,ANTH 199RA,GAMBLE L H,A,1,ANTH
125383,F18,Undergraduate,ANTH 199RA,GURVEN M D,A,4,ANTH
125384,F18,Undergraduate,ANTH 199RA,RESNICK E F,A,2,ANTH
